# Tax-Calculator Demo
This notebook provides a demo of the `Tax-Calculator` micro-simulation model.  The goal is to show some of the inputs and outputs of this model to provide and understanding of what the foundation is for the tax functions that are used in the OG model.

#### Getting Started
To use this notebook, you will need the `Tax-Calculator` package (in addition to a number of packages that will come standard with the Anaconda distribution of Python).  To get `Tax-Calculator`, open your Anaconda command prompt (Windows) or terminal window (Mac).  From here, type

```
conda install -c ospc taxcalc
```

In [ ]:
# imports
from taxcalc import *
from bokeh.io import show, output_notebook, save, output_file
output_notebook()

In [ ]:
# Initiate baseline calculator
rec = Records.cps_constructor()
pol = Policy()
calc1 = Calculator(policy=pol, records=rec)
calc1.advance_to_year(2018)
calc1.calc_all()

In [ ]:
# Look at some of data
df = calc1.dataframe(['s006', 'e00200p', 'e00900'])
df.describe()

## Specifying a reform policy

The baseline in `Tax-Calculator` is a current-law baseline.  You can specify your reforms in a JSON file and, using the read_json_param_objects method, create a dictionary containing both policy reforms and behavioral assumptions
The policy reform and any behavioral assumptions need to be in separate JSON files. Save the policy reforms and the behvaioral assumptions and call them whatever you want and update the snippet below.

In [ ]:
reforms = calc1.read_json_param_objects('2017_law.json', None)
reforms

In [ ]:
# Inititate calculator to apply reforms to
recs2 = Records.cps_constructor()
pol2 = Policy()
pol2.implement_reform(reforms['policy'])
calc2 = Calculator(policy=pol2, records=recs2)
calc2.advance_to_year(2018)
calc2.calc_all()

### Behavioral responses

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [ ]:
# Specify come behavior
revised_behavior = {
            2018: {'_BE_inc': [-0.3]},
            2018: {'_BE_sub': [0.2]}
            }

In [ ]:
behavior = Behavior()
behavior.update_behavior(revised_behavior)

### Revenue Estimates

We can compute a "truly static" revenue estimate by not using this behavior.  Here's an estimate of the revenue under the baseline and policy for one year.

In [ ]:
# calculate reform income tax liabilities for year = cyr
cyr = 2018
calc1.advance_to_year(cyr)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')
calc2.advance_to_year(cyr)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')

# print total revenue estimates for cyr
# (estimates in billons of dollars rounded to nearest hundredth of a billion)
print(cyr, ' Current Law IIT Revenue ($B)= {:.2f}'.format(itax_rev1 * 1e-9))
print(cyr, ' Reform Policy IIT Revenue ($B)= {:.2f}'.format(itax_rev2 * 1e-9))

### Viewing the results

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [ ]:
mtr_plot_data = calc1.mtr_graph(calc2, mars=2,
                                mtr_measure='itax',
                                mtr_variable='e00200p',
                                alt_e00200p_text='Wages and Salaries',
                                mtr_wrt_full_compen=False,
                                income_measure='agi',
                                dollar_weighting=False)
output_file(filename='wage_mtrs.html')
save(mtr_plot_data)
show(mtr_plot_data)

In [ ]:
mtr_plot_data = calc1.mtr_graph(calc2, mars=2,
                                mtr_measure='itax',
                                mtr_variable='e00900p', 
                                alt_e00200p_text='Proprietorship Income',
                                mtr_wrt_full_compen=False,
                                income_measure='agi',
                                dollar_weighting=False)
output_file(filename='SchC_mtrs.html')
save(mtr_plot_data)
show(mtr_plot_data)

### Moving around over the budget window

The above plots were for 2018, the first year we ran the model for.  We can move the model forward in the budget window and look at outcomes in a different year.

In [ ]:
calc1.advance_to_year(2020)
calc2.advance_to_year(2020)

In [ ]:
mtr_plot_data = calc1.mtr_graph(calc2, mars=2,
                                mtr_measure='itax',
                                mtr_variable='e00900p', 
                                alt_e00200p_text='Proprietorship Income',
                                mtr_wrt_full_compen=False,
                                income_measure='agi',
                                dollar_weighting=False)
output_file(filename='SchC_mtrs_2020.html')
save(mtr_plot_data)
show(mtr_plot_data)

In [ ]:
# Plot the average tax rate
atr_plot_data = calc1.atr_graph(calc2, mars=2,
                                atr_measure='combined')
output_file(filename='ATRs_2020.html')
save(atr_plot_data)
show(atr_plot_data)

In [ ]:
fig = calc1.pch_graph(calc2)
show(fig)

### Multiyear difference table
The multiyear differences table function displays the difference between your baseline and refoms. You can group the results by weighted decile, decile, income bin, or age. The default is income bin.

In [ ]:
calc1.diagnostic_table(num_years=2)

In [ ]:
dist1, dist2 = calc1.distribution_tables(calc2, 'weighted_deciles')
dist1

In [ ]:
calc1.mtr('e00200p')


In [ ]:
calc1.mtr('e00200p')[0]-calc2.mtr('e00200p')[0]